# Introducción 

La idea de la práctica es visitar aquellos temas que en cierta manera nos permitan ver más contenido del curso.

La práctica esta dividida en 4 o 5 subapartados, que ya tenéis en este mismo Notebook. Estos subapartados estan aquí para que rellenéis el código que hace falta para la realización de la práctica. Obviamente podéis usar tantas celdas como os hagan falta, es más es de agradecer si el código final esta algo "limpio". Usar funciones, algo de comentario, etc, etc...

Usaremos 2 datasets, uno para el primer ejercicio, y otro para el resto de ejercicios.

Ejercicios:


1.   Machine Learning vs Deep Learning (Acordaros que hay que implementar el pipeline visto en clase entero)

    1.1. Implementación de un modelo de Sentiment Analysis con algún algoritmo de Machine Learning Clásico.
    
    1.2. Implementación de un modelo de Sentiment Analysis con alguna arquitectura de Deep Learning.
    
    1.3. Breve Comparación de resultados. Confusion Matrix.
    
2. Hacer Analysis de los tweets del segundo dataset. Que temas aparecen? Como se representan estos temas? De que hablan unos y otros?

3. Escoged a uno de los dos presidentes, y escribid tweets como ellos, usando un Modelo Generativo.

En cada ejercicio, espero explicaciones y razonamientos del porque una arquitectura y no otra, por ejemplo en Deep Learning, porque usar Convolutionals en lugar de recurrentes, o en Machine Learning, Bayes en lugar de SVM. Hay que explicar el pipeline, sobretodo el preproceso de datos, con lo que habrá que hacer un pequeño estudio de que datos tenemos, y si hay cosas que se pueden ignorar, si hacéis stemming, o no, etc, etc...

Acordaros de que objetivo final no es que obtengáis una accuracy brutal, es que comprendais que pasa cuando usais un algoritmo u otro, y que problemas o beneficios nos dan.

![](https://i.pinimg.com/736x/19/63/8c/19638c0b33e2f7822d6806ce31d89d84--funny-cartoons-funny-jokes.jpg =400x)

Mucha suerte y ánimo!



In [0]:
#Instalaciones necesarias
!pip install spacy
!python -m spacy download en_core_web_md
!pip install networkx==1.11 # para instala hyperopt correctamente, si no, da errores
!pip install hyperopt

In [0]:
import spacy
from google.colab import files
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from keras.layers import Conv1D, Conv2D, SimpleRNN, LSTM, Dense, Dropout
from keras.models import Sequential
from keras.layers import Embedding
from keras.layers import Flatten, Input
from keras.models import Model
from keras.layers import Input, Average, average, Lambda
from keras.models import Model
from keras.optimizers import Adam
from keras import backend as K
from keras.callbacks import EarlyStopping
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
import sys
import time
from keras.layers import MaxPooling1D, GlobalMaxPooling1D
from keras.layers import Input, Embedding, Concatenate
from keras.models import Model
from collections import Counter



nlp = spacy.load('en_core_web_md')

# Dataset

Breve explicación del dataset

# Práctica/Código a entregar

## Pre-train Sentiment Analysis Model

## Sentiment Analysis with Politics

In [4]:
uploaded = files.upload()

Saving train_sentiment_utf8.csv to train_sentiment_utf8.csv


### Realizamos un primer análisis de los textos sin procesarlos

In [0]:
#Cargamos el fichero en un dataframe de pandas para analizarlo
df = pd.read_csv('train_sentiment_utf8.csv')

In [5]:
#Revisamos visualmente el conteniddo del fichero
df.head(10)

,ItemID,Sentiment,SentimentText
0,1,0,is so sad for my APL frie...
1,2,0,I missed the New Moon trail...
2,3,1,omg its already 7:30 :O
3,4,0,.. Omgaga. Im sooo im gunna CRy. I'...
4,5,0,i think mi bf is cheating on me!!! ...
5,6,0,or i just worry too much?
6,7,1,Juuuuuuuuuuuuuuuuussssst Chillin!!
7,8,0,Sunny Again Work Tomorrow :-| ...
8,9,1,handed in my uniform today . i miss you ...
9,10,1,hmmmm.... i wonder how she my number @-)


In [6]:
df['Sentiment'].value_counts()

1    56457
0    43532
Name: Sentiment, dtype: int64

In [0]:
words = set()
for x in df['SentimentText']:
  for w in x.strip().split(' '):
    words.add(w.strip())

In [8]:
len(words)

183641

### Primeras observaciones

Los datos estan balanceados   
Sentiment  
1    56457  
0    43532  

ItemID no nos aporta nada

El lenguaje es ingles, podremos utilizar un modelo de spacy en ingles para procesarlo

Podemos ver que hay muchos caracteres y simbolos de puntuació que eliminaremos para hacerle el trabajo mas facil al algoritmo

Tamaño de nuestro vocabulario 183641 palabras antes de limpiarlo


In [0]:
wordsP = []
for x in df[df['Sentiment']==1]['SentimentText']:
  for w in x.strip().split(' '):
    wordsP.append(w.strip())
    
wordsN = []
for x in df[df['Sentiment']==0]['SentimentText']:
  for w in x.strip().split(' '):
    wordsN.append(w.strip())


In [10]:
# revisamos las palabras mas repetidas en los sentimientos positivos, las que mas aparecen son palabras 
# muy comunes en ambos casos que no nos van a ayudar a diferenciar por eso inspeccionamos un poco mas abajo 
pd.DataFrame(wordsP)[0].value_counts().to_frame()[30:50]

,0
love,2636
at,2395
u,2365
get,2318
not,2311
all,2168
do,2123
can,2058
will,2037
this,1972


In [11]:
pd.DataFrame(wordsN)[0].value_counts().to_frame()[30:50]

,0
are,2022
this,1956
u,1829
all,1826
don't,1822
your,1816
go,1760
up,1733
out,1729
do,1629


En un primer vistazo rapido vemos que palabras componen cada uno de los grupos

### Procesamiento de la entrada
En primer lugar vamos a procesar con spaCy los textos para dividirlo en Tokens, vamos a sacar los lemas para reducir el vocabulario y eiminar signos de puntuación

En este caso vamos a optar por eliminar todas las palabras que no esten en el vocabulario de spacy, la mayoría siguen los siguienes patrones de palabras
* http://myloc.me/27hx   
* @shaunamanu  
* Palabras mal escritas  
* Palabras inexistentes  

Para el caso de las palabras mal scritas se podría utilizar un corrector ortografico tipo hunspell para obtener la palabra mas cercana.






In [0]:
def f(row):
  doc_t = nlp(row['SentimentText'])
  text = ""
  for t in doc_t:
    if not t.is_punct and not t.is_oov:
      text = text + " " + t.lemma_
  return text

df['cleanText'] = df.apply(f, axis=1)

Volvemos a estudiar el texto resultado

In [10]:
df.head(20)

,ItemID,Sentiment,SentimentText,cleanText
0,1,0,is so sad for my APL frie...,be so sad for -PRON- ap...
1,2,0,I missed the New Moon trail...,-PRON- miss the new moon ...
2,3,1,omg its already 7:30 :O,omg -PRON- already 7:30 :o
3,4,0,.. Omgaga. Im sooo im gunna CRy. I'...,-PRON- be sooo -PRON- be gunna -...
4,5,0,i think mi bf is cheating on me!!! ...,i think mi bf be cheat on -PRON- ...
5,6,0,or i just worry too much?,or i just worry too much
6,7,1,Juuuuuuuuuuuuuuuuussssst Chillin!!,chillin
7,8,0,Sunny Again Work Tomorrow :-| ...,sunny again work tomorrow :...
8,9,1,handed in my uniform today . i miss you ...,hand in -PRON- uniform today i miss -P...
9,10,1,hmmmm.... i wonder how she my number @-),hmmmm i wonder how -PRON- -PRON-


In [11]:
words = set()
for x in df['cleanText']:
  for w in x.strip().split(' '):
    words.add(w.strip())
len(words)

33925

Podemos ver que el vocabulario es mucho menor que era

In [0]:
#Guardo y descargo el texto generado para no tener que volver a ejecutarlo
df.to_csv('sentiment.csv')
files.download('sentiment.csv') 


In [2]:
uploaded = files.upload()

Saving sentiment.csv to sentiment.csv


In [0]:
df = pd.read_csv('sentiment.csv')

### Dividimos los datos de entrenamiento en train y test

In [0]:
def split_train_val_test(dataset, split=0.2):

    x, y = zip(*dataset)
    x = np.array(list(x))
    y = np.array(list(y))
    sss = StratifiedShuffleSplit(n_splits=1, test_size=split, random_state=1337) #l33t seed
    for train_index, test_index in sss.split(x, y):
        x_train, x_val = x[train_index], x[test_index]
        y_train, y_val = y[train_index], y[test_index]
    splits = {'train':(x_train, y_train), 'test':(x_val, y_val)}
    return splits

subset = df[['cleanText','Sentiment']]
tuples = [tuple(x) for x in subset.values]
split = split_train_val_test(tuples)

## Entrenamiento de algoritmos

A continuación pasamos a entrenar los algoritmos con los textos procesados  
Vamos a probar 2 algoritmos, Bayes y SVM y trataremos de optimizarlos a ver cual da mejores resultados.

Comenzamos por Bayes

In [0]:
#Preparamos el pipeline de procesamiento
pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('clf', MultinomialNB())
])

In [0]:
parameters = {
    'vect__strip_accents': ['ascii'], #Tartamos los acentos
    'vect__analyzer': ['word'], # Analizamos a nivel de palabra
    'vect__ngram_range': ((1, 3), (2,3) , (1,4)), # Probamos diferentes randos de n-grams
    'vect__stop_words': [None], #Probamos con las stop words por defecto en ingles o sin stop words
    'vect__lowercase': [True], # Se pasa todo a minusculas
    'vect__max_df': (0.80, 0.9, 0.95), # Probamos en un caso a quitar las palabras mas repetidas para hacer un stopr words especifico de este dataset
    'clf__alpha': (1.0, 0.95, 0.90)
}

In [29]:
grid_search = GridSearchCV(pipeline, parameters, verbose=1)
grid_search.fit(split['train'][0], split['train'][1])
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))
print("Best score: %0.3f" % grid_search.best_score_)

# evaluamos en test
pipeline.set_params(**best_parameters)
predictions = pipeline.score(split['test'][0], split['test'][1])
print('TEST SCORE: {}'.format(predictions))

Fitting 3 folds for each of 27 candidates, totalling 81 fits


[Parallel(n_jobs=1)]: Done  81 out of  81 | elapsed: 12.0min finished


	clf__alpha: 1.0
	vect__analyzer: 'word'
	vect__lowercase: True
	vect__max_df: 0.8
	vect__ngram_range: (1, 3)
	vect__stop_words: None
	vect__strip_accents: 'ascii'
Best score: 0.759
TEST SCORE: 0.7608260826082608


[Parallel(n_jobs=1)]: Done  81 out of  81 | elapsed: 12.0min finished   

	clf__alpha: 1.0
	vect__analyzer: 'word'
	vect__lowercase: True
	vect__max_df: 0.8
	vect__ngram_range: (1, 3)
	vect__stop_words: None
	vect__strip_accents: 'ascii'
Best train score: 0.759  
TEST SCORE: 0.7608260826082608

A continuación pasamos a evaluar SVM a ver como se comporta para este caso.   
Incorporamos al pipe un scaler que es recomendable para este algoritmo.  
El problema de SVM es que tarda mucho en entrenarse, por lo que no podemos hacer gran numero de pruebas, por lo que dejamos el kernel rbf, dejamos los mejores parametros obtenidos con el otro algoritmo y probamos con parámetros especificos de este algoritmo

In [0]:


pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('scaler', StandardScaler(with_mean=False)),
    ('svc', SVC())
])

parameters = {
    'vect__strip_accents': ['ascii'], #Tartamos los acentos
    'vect__analyzer': ['word'], # Analizamos a nivel de palabra
    'vect__ngram_range': [(1, 3)], # Probamos diferentes randos de n-grams
    'vect__stop_words': [None], #Probamos con las stop words por defecto en ingles o sin stop words
    'vect__lowercase': [True], # Se pasa todo a minusculas
    'vect__max_df': [0.80], # Probamos en un caso a quitar las palabras mas repetidas para hacer un stopr words especifico de este dataset
    'svc__max_iter': [150, 300],
    'svc__C': np.logspace(-2, 1, 4),
    'svc__gamma':np.logspace(-2, 1, 4),
}

grid_search = GridSearchCV(pipeline, parameters, verbose=1)

grid_search.fit(split['train'][0], split['train'][1])
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))
print("Best score: %0.3f" % grid_search.best_score_)



El resultado es bastante peor al menos con estos parámetros

	svc__C: 0.01
	svc__gamma: 10.0
	svc__max_iter: 300
	vect__analyzer: 'word'
	vect__lowercase: True
	vect__max_df: 0.8
	vect__ngram_range: (1, 3)
	vect__stop_words: None
	vect__strip_accents: 'ascii'
Best score: 0.565

## Modelos deep learning

A continuación pasamos a intentar resolver el problema con modelos de deep learning

Comenzamos preparando las frases, vamos a filtrar las que tengan una logitud grande, que son pocas y provocan una distorsion (outliers)

In [40]:
maxlen = max([len(x.strip()) for x in subset['cleanText']])
maxlen

589

In [42]:
lens = [len(x.strip()) for x in subset['cleanText']]
median = np.median(np.array(lens))
mean = np.mean(np.array(lens))
maxlen = int(median)*2
print(median, mean, maxlen)

59.0 63.5953054836032 118


In [0]:
dataset = subset[['cleanText','Sentiment']].values.tolist()

In [51]:
tokenized_filtered = [(x, y) for x, y in dataset if len(x) < maxlen]
len(tokenized_filtered)

89049

A continuación pasamo a obtener el vocabulario y preparar los input para las redes neuronales

In [55]:
nlp = spacy.load('en_core_web_md', disable=['parser', 'tagger', 'ner', 'textcat'])
tokens = []
tokenized = []
for x, y in tokenized_filtered:
    x_t = nlp(x)
    toks = [t.text for t in x_t]
    tokens+= toks
    tokenized.append((toks, y))
    
vocab_counter = Counter(tokens)
vocab = set(tokens)
print('Num de features a usar: ', len(vocab))  
print(len(tokenized))  

# Construir el vocabulario como siempre
w2id = {k:i for i, k in enumerate(vocab)}
w2id['<UNK>'] = len(w2id)
labels = ['Negative','Positive']
l2id = {label:i for i, label in enumerate(labels)}
# Preparar Input. Padding. Conversión a input
# maxlen = min(maxlen, 50)
input_ready = []
for x, y in tokenized_filtered:
    sentence = np.zeros((maxlen))
    label = np.zeros((len(labels)))
    label[int(y)-1] = 1
    for i, t in enumerate(x):
        sentence[i] = w2id[t] if t in vocab_counter and vocab_counter[t]>=5 else w2id['<UNK>']
    input_ready.append((sentence,label))

Num de features a usar:  30385
89049


Dividimos en train y test

In [77]:
splits = split_train_val_test(input_ready)
print(splits['train'][0].shape)
print(splits['train'][1].shape)
splits['train'][1]

(71239, 118)
(71239, 2)


array([[1., 0.],
       [0., 1.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]])

### Modelo Deep Learning simple
En primer lugar vamos a probar el rendimienno de un modelo simple, solo con capas densas y de drop out a ver como se comporta

In [90]:
model = Sequential()
model.add(Dense(500, input_dim=maxlen, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(300, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(l2id), activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, 50)                5950      
_________________________________________________________________
dropout_9 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_14 (Dense)             (None, 30)                1530      
_________________________________________________________________
dense_15 (Dense)             (None, 2)                 62        
Total params: 7,542
Trainable params: 7,542
Non-trainable params: 0
_________________________________________________________________


In [91]:
model.fit(splits['train'][0], splits['train'][1],
          epochs=5,
          batch_size=64)

Epoch 1/5
71239/71239 [==============================] - 7s 98us/step - loss: 7.0787 - acc: 0.5588
Epoch 2/5
71239/71239 [==============================] - 7s 95us/step - loss: 6.8105 - acc: 0.5756
Epoch 3/5
71239/71239 [==============================] - 7s 95us/step - loss: 6.8021 - acc: 0.5761
Epoch 4/5
37568/71239 [==============>...............] - ETA: 3s - loss: 6.8138 - acc: 0.5754

71239/71239 [==============================] - 7s 94us/step - loss: 6.7970 - acc: 0.5764
Epoch 5/5
71239/71239 [==============================] - 7s 95us/step - loss: 6.7975 - acc: 0.5764


In [89]:
score_basic = model.evaluate(splits['test'][0], splits['test'][1], batch_size=16)
score_basic

17810/17810 [==============================] - 2s 128us/step


[6.790860308133639, 0.5765300393037619]

Podemos observar que este modelo no da un gran rendimiento, vamos a probar con modelos mas complejos

### Modelo basado en Deep Averaging Networks
La primera opción elegida es intentar solucionar el problema con este modelo de red neuronal. Vamos a usar hyperopts para intentar encontrar los mejores parámetros de la red.

In [0]:
def mask_aware_mean(x):
    # recreate the masks - all zero rows have been masked
    mask = K.not_equal(K.sum(K.abs(x), axis=2, keepdims=True), 0)
    # number of that rows are not all zeros
    n = K.sum(K.cast(mask, 'float32'), axis=1, keepdims=False)
    # compute mask-aware mean of x
    x_mean = K.sum(x, axis=1, keepdims=False) / n
    return x_mean

def mask_aware_mean_output_shape(input_shape):
    shape = list(input_shape)
    assert len(shape) == 3
    return (shape[0], shape[2])

In [0]:


space = {
    'dropout': hp.uniform('dropout', 0.0, 0.5),
    'neurons_dense': hp.choice('neurons_dense', [64, 128, 256]), 
    'embeding_dim': hp.choice('embeding_dim', [50, 100, 150]),
    'lr': hp.choice('lr', [0.001, 0.01, 0.1]),
    'act_func' : hp.choice('act_func', ['relu','tanh'])
}

X_train = splits['train'][0]
y_train = splits['train'][1]
## Soy consciente de que debería tener unos datos de validación aparte de los de test, pero por falta de tiempo no o he realizado
X_val = splits['test'][0]
y_val = splits['test'][1]



def	get_callbacks(pars):
  callbacks	= [EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=2, verbose=0, mode='auto')]
  return callbacks

def mi_cnn(pars):
  print ('Parameters: ', pars)
  input_layer = Input(shape=(maxlen,) )
  embedding = Embedding(output_dim=pars['embeding_dim'], input_dim=len(w2id), input_length=maxlen)(input_layer)
  doc_representation = Lambda(mask_aware_mean, mask_aware_mean_output_shape, name='embedding_average')(embedding)
  dense_1 = Dense(pars['neurons_dense'], activation=pars['act_func'])(doc_representation)
  drop_1 = Dropout(pars['dropout'])(dense_1)
  dense_2 = Dense(pars['neurons_dense'], activation=pars['act_func'])(drop_1)
  drop_2 = Dropout(pars['dropout'])(dense_2)
  out = Dense(len(l2id), activation='softmax')(drop_2)

  model = Model(inputs=input_layer, outputs=out)

  model.compile(loss='categorical_crossentropy',
                optimizer=Adam(lr=pars['lr'], decay=1e-6),
                metrics=['accuracy'])
  # Entrenamos el modelo
  history = model.fit(X_train, 
                      y_train,
                      batch_size=128,
                      shuffle=True,
                      epochs=5,
                      validation_data=(X_val, y_val),
                      verbose = 1)

  best_epoch_loss = np.argmin(history.history['val_loss'])
  best_val_loss = np.min(history.history['val_loss'])
  best_val_acc = np.max(history.history['val_acc'])
  
  print('Epoch {} - val acc: {} - val loss: {}'.format(best_epoch_loss, best_val_acc, best_val_loss))
  sys.stdout.flush()
  
  return {'Accuracy': best_val_acc,'loss': best_val_loss, 'best_epoch': best_epoch_loss, 'eval_time': time.time(), 'status': STATUS_OK, 'model': model, 'history': history}


In [134]:
trials = Trials()
best = fmin(mi_cnn, space, algo=tpe.suggest, max_evals=10, trials=trials)
print(best)

Parameters:  {'act_func': 'relu', 'dropout': 0.03949876009208708, 'embeding_dim': 50, 'lr': 0.001, 'neurons_dense': 128}
Train on 71239 samples, validate on 17810 samples
Epoch 1/5
71239/71239 [==============================] - 8s 106us/step - loss: 0.6715 - acc: 0.5893 - val_loss: 0.6685 - val_acc: 0.5906
Epoch 2/5
71239/71239 [==============================] - 6s 85us/step - loss: 0.6589 - acc: 0.6062 - val_loss: 0.6579 - val_acc: 0.6062
Epoch 3/5
33664/71239 [=============>................] - ETA: 3s - loss: 0.6576 - acc: 0.6092

71239/71239 [==============================] - 6s 86us/step - loss: 0.6570 - acc: 0.6093 - val_loss: 0.6598 - val_acc: 0.5999
Epoch 4/5
71239/71239 [==============================] - 6s 85us/step - loss: 0.6552 - acc: 0.6104 - val_loss: 0.6533 - val_acc: 0.6099
Epoch 5/5
71239/71239 [==============================] - 6s 85us/step - loss: 0.6536 - acc: 0.6120 - val_loss: 0.6552 - val_acc: 0.6098
Epoch 3 - val acc: 0.609882088740979 - val loss: 0.6533223645128875
Parameters:  {'act_func': 'tanh', 'dropout': 0.11984438975016393, 'embeding_dim': 100, 'lr': 0.001, 'neurons_dense': 128}
Train on 71239 samples, validate on 17810 samples
Epoch 1/5
24960/71239 [=========>....................] - ETA: 7s - loss: 0.6771 - acc: 0.5803

71239/71239 [==============================] - 9s 123us/step - loss: 0.6696 - acc: 0.5922 - val_loss: 0.6646 - val_acc: 0.6006
Epoch 2/5
71239/71239 [==============================] - 7s 100us/step - loss: 0.6633 - acc: 0.6018 - val_loss: 0.6618 - val_acc: 0.6042
Epoch 3/5
71239/71239 [==============================] - 7s 100us/step - loss: 0.6629 - acc: 0.6006 - val_loss: 0.6608 - val_acc: 0.6047
Epoch 4/5
29440/71239 [===========>..................] - ETA: 3s - loss: 0.6602 - acc: 0.6067

71239/71239 [==============================] - 7s 100us/step - loss: 0.6620 - acc: 0.6031 - val_loss: 0.6607 - val_acc: 0.6053
Epoch 5/5
71239/71239 [==============================] - 7s 99us/step - loss: 0.6620 - acc: 0.6027 - val_loss: 0.6607 - val_acc: 0.6012
Epoch 3 - val acc: 0.6053340819764177 - val loss: 0.6606554592612887
Parameters:  {'act_func': 'relu', 'dropout': 0.4166948837224789, 'embeding_dim': 50, 'lr': 0.1, 'neurons_dense': 256}
Train on 71239 samples, validate on 17810 samples
Epoch 1/5
71239/71239 [==============================] - 8s 111us/step - loss: 0.7023 - acc: 0.5747 - val_loss: 0.6835 - val_acc: 0.5765
Epoch 2/5
  128/71239 [..............................] - ETA: 7s - loss: 0.6777 - acc: 0.6094

71239/71239 [==============================] - 6s 85us/step - loss: 0.6835 - acc: 0.5757 - val_loss: 0.6824 - val_acc: 0.5765
Epoch 3/5
71239/71239 [==============================] - 6s 86us/step - loss: 0.6834 - acc: 0.5756 - val_loss: 0.6831 - val_acc: 0.5765
Epoch 4/5
71239/71239 [==============================] - 6s 86us/step - loss: 0.6835 - acc: 0.5766 - val_loss: 0.6815 - val_acc: 0.5765
Epoch 5/5
36608/71239 [==============>...............] - ETA: 2s - loss: 0.6831 - acc: 0.5767

...

71239/71239 [==============================] - 7s 102us/step - loss: 6.8252 - acc: 0.5766 - val_loss: 6.8255 - val_acc: 0.5765
Epoch 0 - val acc: 0.5765300393071087 - val loss: 6.8255292845334585
Parameters:  {'act_func': 'relu', 'dropout': 0.14992775094124322, 'embeding_dim': 150, 'lr': 0.1, 'neurons_dense': 64}
Train on 71239 samples, validate on 17810 samples
Epoch 1/5
71239/71239 [==============================] - 10s 147us/step - loss: 0.6944 - acc: 0.5752 - val_loss: 0.6814 - val_acc: 0.5765
Epoch 2/5
23680/71239 [========>.....................] - ETA: 5s - loss: 0.6821 - acc: 0.5793

71239/71239 [==============================] - 8s 113us/step - loss: 0.6829 - acc: 0.5766 - val_loss: 0.6835 - val_acc: 0.5765
Epoch 3/5
71239/71239 [==============================] - 8s 113us/step - loss: 0.6831 - acc: 0.5766 - val_loss: 0.6820 - val_acc: 0.5765
Epoch 4/5
71239/71239 [==============================] - 8s 112us/step - loss: 0.6829 - acc: 0.5749 - val_loss: 0.6826 - val_acc: 0.5765
Epoch 5/5
10880/71239 [===>..........................] - ETA: 6s - loss: 0.6845 - acc: 0.5797

71239/71239 [==============================] - 8s 112us/step - loss: 0.6836 - acc: 0.5759 - val_loss: 0.6829 - val_acc: 0.5765
Epoch 0 - val acc: 0.5765300393071087 - val loss: 0.681387596234788
Parameters:  {'act_func': 'relu', 'dropout': 0.1604723930901929, 'embeding_dim': 100, 'lr': 0.01, 'neurons_dense': 64}
Train on 71239 samples, validate on 17810 samples
Epoch 1/5
71239/71239 [==============================] - 10s 134us/step - loss: 0.6720 - acc: 0.5843 - val_loss: 0.6654 - val_acc: 0.5779
Epoch 2/5
32640/71239 [============>.................] - ETA: 3s - loss: 0.6648 - acc: 0.5969

71239/71239 [==============================] - 7s 102us/step - loss: 0.6631 - acc: 0.5991 - val_loss: 0.6654 - val_acc: 0.6002
Epoch 3/5
71239/71239 [==============================] - 7s 102us/step - loss: 0.6613 - acc: 0.6039 - val_loss: 0.6622 - val_acc: 0.6062
Epoch 4/5
71239/71239 [==============================] - 7s 102us/step - loss: 0.6597 - acc: 0.6040 - val_loss: 0.6566 - val_acc: 0.6058
Epoch 5/5
26752/71239 [==========>...................] - ETA: 4s - loss: 0.6595 - acc: 0.6041

71239/71239 [==============================] - 7s 102us/step - loss: 0.6584 - acc: 0.6051 - val_loss: 0.6602 - val_acc: 0.6054
Epoch 3 - val acc: 0.606176305439685 - val loss: 0.6566270524902601
{'act_func': 0, 'dropout': 0.03949876009208708, 'embeding_dim': 0, 'lr': 0, 'neurons_dense': 1}


In [153]:
i=0
max = 0
for c, x in enumerate(trials.results):
  if(x['Accuracy'] > max):
    max = x['Accuracy']
    i = c
(i, max)
trials.trials[i]  



{'book_time': datetime.datetime(2018, 7, 20, 19, 28, 0, 259000),
 'exp_key': None,
 'misc': {'cmd': ('domain_attachment', 'FMinIter_Domain'),
  'idxs': {'act_func': [5],
   'dropout': [5],
   'embeding_dim': [5],
   'lr': [5],
   'neurons_dense': [5]},
  'tid': 5,
  'vals': {'act_func': [0],
   'dropout': [0.019778751974348807],
   'embeding_dim': [1],
   'lr': [0],
   'neurons_dense': [2]},
  'workdir': None},
 'owner': None,
 'refresh_time': datetime.datetime(2018, 7, 20, 19, 28, 39, 278000),
 'result': {'Accuracy': 0.6115665356474335,
  'best_epoch': 4,
  'eval_time': 1532114919.2780857,
  'history': <keras.callbacks.History at 0x7f55c47bba90>,
  'loss': 0.6538323769001797,
  'model': <keras.engine.training.Model at 0x7f55c51c1c50>,
  'status': 'ok'},
 'spec': None,
 'state': 2,
 'tid': 5,
 'version': 0}

Estos son los mejores resultados optenidos y los parametros utilizados para ello

   >('dropout', 0.03),  
   >('neurons_dense', [128]),   
   >('embeding_dim', [50,]),  
   >('lr', [0.001]),   
   >('act_func', ['relu'])  
   
   'Accuracy': 0.6115665356474335
   
   Hemos mejorado el rendimiento respecto a la red neuronal simple, pero aun esta lejos de los resultados obtenidos con bayes.

### Modelo basado en redes neuronales convolucionales
A continuación vamos a probar un modelos de este tipo, tambien vamos a usar hyperopt para intenar optimizar los parámetros de la red.

In [0]:
space = {
    'dropout': hp.uniform('dropout', 0.0, 0.5),
    'neurons_dense': hp.choice('neurons_dense', [64, 128, 256]), 
    'embeding_dim': hp.choice('embeding_dim', [50, 100, 150]),
    'filters': hp.choice('filters', [50, 100, 150]),
}

X_train = splits['train'][0]
y_train = splits['train'][1]
## Soy consciente de que debería tener unos datos de validación aparte de los de test, pero por falta de tiempo no o he realizado
X_val = splits['test'][0]
y_val = splits['test'][1]



def	get_callbacks(pars):
  callbacks	= [EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=2, verbose=0, mode='auto')]
  return callbacks

def mi_cnn(pars):
  print ('Parameters: ', pars)

  input_layer = Input(shape=(maxlen,) )# maxlen
  embedding = Embedding(output_dim=pars['embeding_dim'], input_dim=len(w2id), input_length=maxlen)(input_layer)#w2id maxlen

  conv_1 = Conv1D(filters=pars['filters'], kernel_size=1, strides=1, activation='relu', padding='valid')(embedding)
  mp_1 = GlobalMaxPooling1D()(conv_1)

  conv_2 = Conv1D(filters=pars['filters'], kernel_size=2, strides=1, activation='relu', padding='valid')(embedding)
  mp_2 = GlobalMaxPooling1D()(conv_2)

  conv_5 = Conv1D(filters=pars['filters'], kernel_size=5, strides=1, activation='relu', padding='valid')(embedding)
  mp_5 = GlobalMaxPooling1D()(conv_5)

  doc_representation = Concatenate()([mp_1, mp_2, mp_5])

  dense_1 = Dense(pars['neurons_dense'], activation='relu')(doc_representation)
  drop_1 = Dropout(pars['dropout'])(dense_1)
  dense_2 = Dense(pars['neurons_dense'], activation='relu')(drop_1)
  drop_2 = Dropout(pars['dropout'])(dense_2)
  out = Dense(len(l2id), activation='softmax')(drop_2)

  model = Model(inputs=input_layer, outputs=out)

  model.compile(loss='categorical_crossentropy',
                optimizer='rmsprop',
                metrics=['accuracy'])

  # Entrenamos el modelo
  history = model.fit(X_train, 
                      y_train,
                      batch_size=256,
                      shuffle=True,
                      epochs=10,
                      validation_data=(X_val, y_val),
                      verbose = 1)

  best_epoch_loss = np.argmin(history.history['val_loss'])
  best_val_loss = np.min(history.history['val_loss'])
  best_val_acc = np.max(history.history['val_acc'])
  
  print('Epoch {} - val acc: {} - val loss: {}'.format(best_epoch_loss, best_val_acc, best_val_loss))
  sys.stdout.flush()
  
  return {'Accuracy': best_val_acc,'loss': best_val_loss, 'best_epoch': best_epoch_loss, 'eval_time': time.time(), 'status': STATUS_OK, 'model': model, 'history': history}


In [162]:
trials = Trials()
best = fmin(mi_cnn, space, algo=tpe.suggest, max_evals=10, trials=trials)
print(best)

Parameters:  {'dropout': 0.2573870381619406, 'embeding_dim': 50, 'filters': 100, 'neurons_dense': 256}
Train on 71239 samples, validate on 17810 samples
Epoch 1/10
71239/71239 [==============================] - 13s 177us/step - loss: 0.6209 - acc: 0.6471 - val_loss: 0.7337 - val_acc: 0.6147
Epoch 2/10
71239/71239 [==============================] - 8s 106us/step - loss: 0.5581 - acc: 0.7068 - val_loss: 0.5383 - val_acc: 0.7289
Epoch 3/10
16896/71239 [======>.......................] - ETA: 5s - loss: 0.5374 - acc: 0.7225

71239/71239 [==============================] - 8s 107us/step - loss: 0.5328 - acc: 0.7270 - val_loss: 0.5393 - val_acc: 0.7358
Epoch 4/10
71239/71239 [==============================] - 8s 107us/step - loss: 0.5159 - acc: 0.7398 - val_loss: 0.5027 - val_acc: 0.7525
Epoch 5/10
71239/71239 [==============================] - 8s 106us/step - loss: 0.5050 - acc: 0.7478 - val_loss: 0.5079 - val_acc: 0.7518
Epoch 6/10
17152/71239 [======>.......................] - ETA: 5s - loss: 0.4993 - acc: 0.7561

71239/71239 [==============================] - 8s 106us/step - loss: 0.4959 - acc: 0.7545 - val_loss: 0.5398 - val_acc: 0.7207
Epoch 7/10
71239/71239 [==============================] - 8s 107us/step - loss: 0.4865 - acc: 0.7606 - val_loss: 0.5374 - val_acc: 0.7323
Epoch 8/10
71239/71239 [==============================] - 8s 106us/step - loss: 0.4793 - acc: 0.7660 - val_loss: 0.4945 - val_acc: 0.7568
Epoch 9/10
15104/71239 [=====>........................] - ETA: 5s - loss: 0.4704 - acc: 0.7719

71239/71239 [==============================] - 8s 106us/step - loss: 0.4730 - acc: 0.7689 - val_loss: 0.5439 - val_acc: 0.7423
Epoch 10/10
71239/71239 [==============================] - 8s 106us/step - loss: 0.4673 - acc: 0.7739 - val_loss: 0.4978 - val_acc: 0.7595
Epoch 7 - val acc: 0.7595171253176501 - val loss: 0.4944729449607093
Parameters:  {'dropout': 0.11644216231957516, 'embeding_dim': 50, 'filters': 50, 'neurons_dense': 64}
Train on 71239 samples, validate on 17810 samples
Epoch 1/10
71239/71239 [==============================] - 11s 156us/step - loss: 0.6209 - acc: 0.6499 - val_loss: 0.6491 - val_acc: 0.6034
Epoch 2/10
 5632/71239 [=>............................] - ETA: 5s - loss: 0.5754 - acc: 0.6955

71239/71239 [==============================] - 6s 83us/step - loss: 0.5645 - acc: 0.7026 - val_loss: 0.5375 - val_acc: 0.7229
Epoch 3/10
71239/71239 [==============================] - 6s 83us/step - loss: 0.5414 - acc: 0.7208 - val_loss: 0.5217 - val_acc: 0.7367
Epoch 4/10
71239/71239 [==============================] - 6s 83us/step - loss: 0.5248 - acc: 0.7335 - val_loss: 0.5752 - val_acc: 0.6876
Epoch 5/10
58624/71239 [=======================>......] - ETA: 0s - loss: 0.5126 - acc: 0.7422

...

71239/71239 [==============================] - 7s 105us/step - loss: 0.4779 - acc: 0.7653 - val_loss: 0.5045 - val_acc: 0.7534
Epoch 9/10
71239/71239 [==============================] - 7s 105us/step - loss: 0.4714 - acc: 0.7690 - val_loss: 0.5132 - val_acc: 0.7526
Epoch 10/10
71239/71239 [==============================] - 7s 105us/step - loss: 0.4663 - acc: 0.7740 - val_loss: 0.5207 - val_acc: 0.7432
Epoch 7 - val acc: 0.7533969680088949 - val loss: 0.5045455954571509
Parameters:  {'dropout': 0.2808516902502133, 'embeding_dim': 100, 'filters': 100, 'neurons_dense': 128}
Train on 71239 samples, validate on 17810 samples
Epoch 1/10
12544/71239 [====>.........................] - ETA: 23s - loss: 0.6701 - acc: 0.5910

71239/71239 [==============================] - 14s 190us/step - loss: 0.6154 - acc: 0.6557 - val_loss: 0.6444 - val_acc: 0.6145
Epoch 2/10
71239/71239 [==============================] - 10s 137us/step - loss: 0.5534 - acc: 0.7126 - val_loss: 0.5363 - val_acc: 0.7357
Epoch 3/10
71239/71239 [==============================] - 10s 137us/step - loss: 0.5323 - acc: 0.7280 - val_loss: 0.5199 - val_acc: 0.7337
Epoch 4/10
 2816/71239 [>.............................] - ETA: 8s - loss: 0.4989 - acc: 0.7429

71239/71239 [==============================] - 10s 138us/step - loss: 0.5161 - acc: 0.7411 - val_loss: 0.5035 - val_acc: 0.7516
Epoch 5/10
71239/71239 [==============================] - 10s 137us/step - loss: 0.5055 - acc: 0.7494 - val_loss: 0.5654 - val_acc: 0.6979
Epoch 6/10
71239/71239 [==============================] - 10s 136us/step - loss: 0.4948 - acc: 0.7550 - val_loss: 0.4967 - val_acc: 0.7492
Epoch 7/10
 1792/71239 [..............................] - ETA: 8s - loss: 0.4674 - acc: 0.7684

71239/71239 [==============================] - 10s 136us/step - loss: 0.4864 - acc: 0.7612 - val_loss: 0.5107 - val_acc: 0.7514
Epoch 8/10
71239/71239 [==============================] - 10s 136us/step - loss: 0.4765 - acc: 0.7667 - val_loss: 0.5871 - val_acc: 0.6967
Epoch 9/10
71239/71239 [==============================] - 10s 136us/step - loss: 0.4700 - acc: 0.7734 - val_loss: 0.5328 - val_acc: 0.7349
Epoch 10/10
  256/71239 [..............................] - ETA: 10s - loss: 0.5009 - acc: 0.7617

71239/71239 [==============================] - 10s 136us/step - loss: 0.4642 - acc: 0.7761 - val_loss: 0.4951 - val_acc: 0.7583
Epoch 9 - val acc: 0.758281863980719 - val loss: 0.4950568249625881
{'dropout': 0.4717708852949422, 'embeding_dim': 0, 'filters': 2, 'neurons_dense': 1}


In [163]:
i=0
max = 0
for c, x in enumerate(trials.results):
  if(x['Accuracy'] > max):
    max = x['Accuracy']
    i = c
print((i, max))
print(trials.trials[i])  

(2, 0.7613138684725774)
{'state': 2, 'tid': 2, 'spec': None, 'result': {'Accuracy': 0.7613138684725774, 'loss': 0.5036781453102376, 'best_epoch': 8, 'eval_time': 1532116766.3420596, 'status': 'ok', 'model': <keras.engine.training.Model object at 0x7f55b3b0af98>, 'history': <keras.callbacks.History object at 0x7f55b3b1e128>}, 'misc': {'tid': 2, 'cmd': ('domain_attachment', 'FMinIter_Domain'), 'workdir': None, 'idxs': {'dropout': [2], 'embeding_dim': [2], 'filters': [2], 'neurons_dense': [2]}, 'vals': {'dropout': [0.4305750314410428], 'embeding_dim': [1], 'filters': [1], 'neurons_dense': [1]}}, 'exp_key': None, 'owner': None, 'version': 0, 'book_time': datetime.datetime(2018, 7, 20, 19, 57, 43, 720000), 'refresh_time': datetime.datetime(2018, 7, 20, 19, 59, 26, 342000)}


Los resultados obtenidos con esta red son ya mejores, muy similares a los obtenidos con bayes. Se presentan tambien los parametros utilizados para obtener estos resultados

'Accuracy': 0.7613138684725774,

{'dropout': 0.4717708852949422, 'embeding_dim': 50, 'filters': 100, 'neurons_dense': 128}
 
 


## Conclusiones
Se ha intentado optimizar 4 algoritmos, 2 clásicos y 2 basados en redes neuronales.

Entre os algoritmos clásicos, hemos utilizado Bayes y SVM, en ambos casos hemos utilizado grid search para intentar encontrar los parámetros optimos. para este caso, se comporta claramente mejor bayes llegando a un accuracy en test del 76%  
TEST SCORE: 0.7608260826082608

Entre los algoritmos de deep learning, hemos probado 2 arquitecturas diferentes, una basada en Deep Averaging network, y a otra basada en convolutional neural networks. En ambos casos se han seguido las estructuras construidas en clase, y se han intentado optimizar los parámetros utilizando hyperopts. Entre estas 2 redes, la que mejor se comporta es la basada en CNN, donde se ha llegado a un accuracy del 76%  para el conjunto de test  
'TEST SCORE:': 0.7613138684725774

Hay muy poca diferencia entre el comportamiento de ambos algoritmos.

